# Function Calling (OpenAI)

[Openai Tutorioal](https://platform.openai.com/docs/guides/function-calling)

### Define the function callback

The function returns the temperature at a location as loat. It expects precise latitude and longitude instead of a general location parameter. However, the model can automatically determine the coordinates.

In [7]:
import requests


def get_weather(latitude: float, longitude: float) -> float:
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,wind_speed_10m"
        f"&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m",
        timeout=10,
    )
    data = response.json()
    return data["current"]["temperature_2m"]

weather = get_weather(48.8566, 2.3522)
print(f"The weather in Paris is {weather}°C")

The weather in Paris is 19.0°C


In [13]:

import json

from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"},
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False,
        },
        "strict": True,
    },
}]

messages = [{"role": "user", "content": "What's the weather like in Zurich today?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion.to_json())

{
  "id": "chatcmpl-B8saMnOMRRtL06IDMFEKzoCFAXvUd",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_uEXup1k2HIhJ2xtlDBRsggDz",
            "function": {
              "arguments": "{\"latitude\":47.3769,\"longitude\":8.5417}",
              "name": "get_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1741456150,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_eb9dce56a8",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 59,
    "total_tokens": 84,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens

In [15]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

print(f"The weather in Zurich is {result}°C")

The weather in Zurich is 12.4°C


In [ ]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result),
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion_2.to_json())

{
  "id": "chatcmpl-B8tIzbuBzz6IZPZWOjaYfnB6qa09I",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The current temperature in Zurich is 12.4°C.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741458917,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_eb9dce56a8",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 129,
    "total_tokens": 143,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
